In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('/content/food_test.csv')

data.head()

,foodId,Nama,Tipe 1,Tipe 2,Tipe 3
0,1,Sosis Bakar,ayam,daging,daging
1,2,Ngohiong Ayam Udang,ayam,daging,daging
2,3,Rawon Ayam,ayam,daging,daging
3,4,Usus Goreng Crispy,ayam,daging,daging
4,5,Ceker Rica Rica,ayam,daging,daging


In [3]:
# preprocessing the dataset
data['Tipe'] = data['Tipe 1'] + ',' + data['Tipe 2'] + ',' + data['Tipe 3']
data['Tipe'] = data['Tipe'].str.lower() 

In [4]:
# Vectorizing pre-processed food type plots using TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['Tipe'])

In [5]:
# Finding cosine similarity between vectors
similarity_matrix = cosine_similarity(tfidf_matrix)
similarity_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [6]:
data.iloc[np.argmax(similarity_matrix)]

foodId                     1
Nama             Sosis Bakar
Tipe 1                  ayam
Tipe 2                daging
Tipe 3                daging
Tipe      ayam,daging,daging
Name: 0, dtype: object

In [8]:
# define recomendation
def get_recommendations_by_types(food_types, similarity_matrix, k=10):
    food_indices = []
    for food_type in food_types:
        indices = data[data['Tipe'].str.contains(food_type)].index
        food_indices.extend(indices)
    food_indices = list(set(food_indices))
    
    similarity_scores = list(enumerate(similarity_matrix[food_indices].sum(axis=0)))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_k_food_indices = [i for i, _ in similarity_scores[1:k+1]]
    top_k_food_names = data.loc[top_k_food_indices, 'Nama'].values
    return top_k_food_names

In [11]:
user_food_types = input("Masukkan tipe makanan (pisahkan dengan koma): ").lower().split(",")
recommendations = get_recommendations_by_types(user_food_types, similarity_matrix)
print(f"Rekomendasi makanan berdasarkan tipe {', '.join(user_food_types)}:")
for i, food in enumerate(recommendations):
    print(f"{i+1}. {food}")

Masukkan tipe makanan (pisahkan dengan koma): puding
Rekomendasi makanan berdasarkan tipe puding:
1. Panna Cotta Strawberry
2. Puding Alpukat
3. Butterscotch Pudding
4. Puding Mozaik
5. Kembang Tahu Wedang (Tahok)
6. Puding Tape Ketan
7. Puding Coklat Susu
8. Puding Lumut
9. Puding Buah Naga
10. Puding Kelapa Muda
